Задания к [первой практике](https://colab.research.google.com/drive/1AKldHA9Ib8eQk4yn3yAi-Ffs4EtatPsh?usp=sharing).

### Задание 1



Сбор данных из ИСУ.

Выберите какой-либо факультет, соберите данные по обучающимся на нём группам бакалавров на всех курсах (ИСУ - Учебный процесс - Учебные группы и потоки). Сформируйте таблицу с колонками `группа`, `курс`, `номер_ису`, `фио`. Ответьте на вопросы:

1. Одинаковое ли количество групп от курса к курсу?

2. Есть ли в вашей выборке однофамильцы (для простоты учитывайте только полное совпадение фамилии)? Если да, выведите этих людей.

3. Есть ли в вашей выборке ребята без отчества? Если да, выведите этих людей.

4. Найдите курс, на котором меньше всего обучающихся. Выведите значение и номер курса. Если таких курсов несколько, выведите их все.

5. Для каждого курса найдите группу, в которой больше всего обучающихся. Выведите тройки (курс, группа, кол-во обучающихся). Если для какого-либо курса таких групп окажется несколько, выведите их все.

Старайтесь писать код так, чтобы при изменении наименования факультета ничего не ломалось.

In [ ]:
# Это необходимо, чтобы запустить код в Google Colab. Источник: https://nariyoo.com/python-how-to-run-selenium-in-google-colab/
%%capture
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium chromedriver-autoinstaller

In [ ]:
import sys
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import chromedriver_autoinstaller


sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--verbose')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--window-size=1920, 1200')
chrome_options.add_argument('--disable-dev-shm-usage')

chromedriver_autoinstaller.install()
driver = webdriver.Chrome(options=chrome_options)  # создание объекта драйвера

In [ ]:
# вход в ИСУ
driver.get("https://id.itmo.ru/login")

# подождать, пока подгрузится страница авторизации
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "username"))  # если случится TimeoutException - перезапустите ячейку
)

<selenium.webdriver.remote.webelement.WebElement (session="20fc30daf2ccacbc7df760ddc6a619eb", element="f.473F4FAC8054978C169BCF989CD15DD1.d.CADB217F3CC96F018289AC163400871A.e.11")>

In [ ]:
# информация для входа в ИСУ
username = ''
password = ''

# ввод данных для входа
username_field = driver.find_element(By.NAME, "username")
password_field = driver.find_element(By.NAME, "password")

username_field.send_keys(username)
password_field.send_keys(password)
password_field.send_keys(Keys.RETURN)

In [ ]:
target_faculty_name = 'факультет инфокоммуникационных технологий'
driver.get('https://isu.ifmo.ru')
# driver.page_source - чтобы увидеть содержимое текущей страницы
link_to_group_page = driver.find_element('xpath', "//a[contains(., 'Учебные группы и потоки')]")
group_url = link_to_group_page.get_attribute('href')

driver.get(group_url)
soup = BeautifulSoup(driver.page_source, 'lxml')

check = False
sample_url = 'https://isu.ifmo.ru/pls/apex/'
df = pd.DataFrame(columns=["группа","курс","номер_ису","фио"])
line = [None, None, None, None]

for el in soup.find_all():

  if (el.get('class') == ['text-black'] and ('факультет' in el.text)) or el.get('class') == ['group-object'] or (el.get('class') == ['no-margin'] and ('курс' in el.text)):

    if 'курс' in el.text:
      line[1] = el.text[0]

    if f'{target_faculty_name}' in el.text:
      check = True

    elif el.get('class') == ['text-black'] and f'{target_faculty_name}' not in el.text:
      check = False

    if check == True and el.get('class') == ['group-object']:
      driver.get(sample_url + el.get('href'))

      group_page = BeautifulSoup(driver.page_source, 'lxml')
      line[0] = el.text

      check2 = 0
      for gr in group_page.find_all('td'):
        if gr.get('data-order-id') == '2' and 'МФ' not in str(gr.text):
          check2 += 1
          line[2] = gr.text

        elif gr.get('data-order-id') == '4' and len(str(gr.text)) > 6:
          check2 +=1
          line[3] = gr.text
        if check2 == 2:
          df = pd.concat([df, pd.DataFrame([line], columns = ["группа","курс","номер_ису","фио"])], ignore_index=True)
          check2 = 0

  if el.text == 'Магистратура' and 'h4' in str(el):
    break
display(df)








,группа,курс,номер_ису,фио
0,K3120,1,375250,Арройо Ариас Николас Рафаэль
1,K3120,1,408374,Вечеренко Анастасия Дмитриевна
2,K3120,1,408637,Закоурцев Андрей Вадимович
3,K3120,1,408655,Зенин Данил Дмитриевич
4,K3120,1,408672,Зубов Алексей Андреевичч
...,...,...,...,...
842,K34422,4,311409,Петросян Анна Мнацакановна
843,K34422,4,314497,Сарновская Мишель Игоревна
844,K34422,4,284418,Тостановская Анна Евгеньевна
845,K34422,4,313037,Третьякова Екатерина Сергеевна


In [ ]:
def change_value(value):
  value = value.split()[0]
  return value

task_1 = df.groupby(['курс'])['группа'].nunique().reset_index(name='количество')
print('1) \n', task_1, '\n')
if task_1['количество'].nunique() == 1:
  print('кол-во групп одинаковое от курса к курсу \n')
elif task_1['количество'].nunique():
  print('кол-во групп разное от курса к курсу \n')


task_2 = {}
count = 0
print('2) \n')
for value in df['фио']:
  if value.split()[0] not in task_2:
    task_2[value.split()[0]] = [value]
  else: task_2[value.split()[0]].append(value)
for key in task_2.keys():
  if len(task_2[key]) >= 2:
    for elem in task_2[key]:
      print(elem)
      count += 1
print(f'Количество однофамильцев: {count}')


count = 0
print('3) \n')
for value in df['фио']:
  if len(value.split()) == 2 or (value.split()[2][-2:] != 'ич' and value.split()[2][-2:] != 'на'):
    print(value, '\n')
    count += 1
print(f'Кол-во людей без отчества: {count} \n')


task_4 = df.groupby('курс').size()
min_index = task_4.idxmin()
print("4) Курс с наименьшим количеством обучающихся:", min_index, '\n')
print("количество:", task_4[min_index] )



task_5={}
table = df.groupby(['группа', 'курс']).size().reset_index(name='количество')
table_max = table.groupby('курс')['количество'].max()
print('5)\n')
for i, j in table_max.items():
  group = table[(table['курс'] == i) & (table['количество'] == j)]
  task_5 = {'курс': i}
  for index, row in group.iterrows():
    task_5['группа'] = row['группа']
    task_5['кол-во'] = row['количество']
  print(task_5)
  task_5 = 0











1) 
   курс  количество
0    1          11
1    2          10
2    3          10
3    4           9 

кол-во групп разное от курса к курсу 

2) 

Игнатьев Алексей Юрьевич
Игнатьев Алексей Александрович
Олойеде Абдуллах Олувасеун
Олойеде Джибрил Болуватифе
Панин Дмитрий Владимирович
Панин Михаил Павлович
Родионов Анатолий Александрович
Родионов Семён Константинович
Селезнев Иван Александрович
Селезнев Илья Олегович
Данилов Назар Олегович
Данилов Владислав Андреевич
Корчагин Роман Павлович
Корчагин Вадим Сергеевич
Лапшина Юлия Сергеевна
Лапшина Екатерина Алексеевна
Иванов Семён Алексеевич
Иванов Виктор Сергеевич
Иванов Владимир Анатольевич
Костенников Павел Юрьевич
Костенников Данил Вячеславович
Новиков Николай Викторович
Новиков Алексей Дмитриевич
Никифоров Максим Александрович
Никифоров Савелий Денисович
Смирнов Георгий Валерьевич
Смирнов Игорь Иванович
Смирнов Кирилл Андреевич
Смирнов Тимур Олегович
Анисимов Владислав Андреевич
Анисимов Степан Дмитриевич
Богданов Максим Александрович


### Задание 2

Одни из самых ужасных с точки зрения формата представления данные - это данные на сайтах государственных организаций (поликлиники, школы и т.п.). Потренируемся работать с подобными сайтами. Соберем данные за муниципальный этап Всероссийской олимпиады школьников [2022/2023 года по Томской области](http://rcro.tomsk.ru/munitsipal-ny-j-e-tap-vsosh-2022-23-arhiv/).

БОльшая часть кода уже написана за вас, однако вам необходимо дополнить функционал по непосредственному извлечению pandas-датафрейма из файлов различного формата (`pdf`, `xlsx`, `docx`), а также посчитать статистические показатели по собранным данным.

Изучите данные на сайте - таблицы имеют схожую, хоть и слегка различающуюся, структуру. Приведем извлеченную из каждого файла таблицу к датафрейму с колонками `предмет`, `класс`, `фио`, `баллы`. Если что-либо отсутствует - заполним пустой строкой, если речь идёт не о баллах, иначе нулем. ФИО можно расматривать как склейку через пробел полей "Фамилия", "Имя" и "Отчество" в случае наличия последнего.

Обратите внимание, что вы можете менять любую часть кода, добавлять свои функции, а также произвести парсинг полностью вручную (хотя это нежелательно).

In [ ]:
!pip install python-docx xls2xlsx img2table[easyocr]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 398.5/398.5 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.9/25.9 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 69.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 50.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 66.9 MB/s eta 0:00:00
  Attempting uninstall: pol

In [ ]:
from img2table.document import PDF
from img2table.ocr import EasyOCR
from xls2xlsx import XLS2XLSX
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import numpy as np
from docx import Document


easyocr = EasyOCR(lang=["ru"], kw={"gpu": False})  # можно использовать любой другой OCR-движок

In [ ]:
def get_file_extension(filename):
    ext = filename.split('.')[-1]
    if ext not in ['xls', 'xlsx', 'pdf', 'docx']:
        raise ValueError
    return ext


def download_file(url, destination):
    response = requests.get(url)
    if response.status_code == 200:
        with open(destination, 'wb') as f:
            f.write(response.content)
        print(f"File downloaded successfully to '{destination}'")
    else:
        print(f"Failed to download file from '{url}'")


def convert_xls_to_xlsx(filename):  # конвертация xlsx-файлов устаревшего образца
    x2x = XLS2XLSX(filename)
    x2x.to_xlsx(filename + "x")
    return filename


def extract_data(subject, grade, filename, ext, verbose=True):
    if ext == 'xlx':
        ext = 'xlsx'
        filename = convert_xls_to_xlsx(filename)

    if ext == 'xlsx':
        dfp = parse_xlsx(filename)

    elif ext == 'pdf':
        extracted_tables = parse_pdf(filename)
        dfp_arr = []
        for i in range(len(extracted_tables)):
          dfp_arr.append(extracted_tables[i][0].df)

        dfp = pd.concat(dfp_arr, ignore_index=True)
        new_header = dfp.iloc[0]
        dfp = dfp[1:]
        dfp.columns = new_header

    elif ext == 'docx':
        tables = parse_docx(filename)
        for i in range(len(tables)):
          table = tables[i]
          data = [[cell.text for cell in row.cells] for row in table.rows]
          dfp = pd.DataFrame(data)
        new_header = dfp.iloc[0]
        dfp = dfp[1:]
        dfp.columns = new_header

    # PUT YOUR CODE HERE - приведение разнородных данных к одному виду
    df = pd.DataFrame(columns=["предмет","класс","фио","баллы"])
    if dfp['Фамилия'].isna().any:
        dfp['Фамилия'] = dfp['Фамилия'].fillna('')
    if dfp['Имя'].isna().any:
        dfp['Имя'] = dfp['Имя'].fillna('')

    df['фио'] = dfp['Фамилия'] + ' ' + dfp['Имя']
    if "Отчество" in dfp:
      if dfp['Отчество'].isna().any:
        dfp['Отчество'] = dfp['Отчество'].fillna('')
      df['фио'] = df['фио']+ ' ' + dfp['Отчество']




    if 'Балл' in dfp:
      df['баллы'] = dfp['Балл']
    elif 'Результат' in dfp:
      df['баллы'] = dfp['Результат']
    elif 'Результ\nат' in dfp:
      df['баллы'] = dfp['Результ\nат']



    df['класс'] = grade

    df['предмет'] = subject

    if verbose:
        print()
        display(df)
        print()

    return df[['предмет', 'класс', 'фио', 'баллы']]


def parse_xlsx(filename):
    dfp = pd.read_excel(filename, skiprows=3)

    return dfp


def parse_docx(filename):
    document = Document(filename)
    return document.tables


def parse_pdf(filename):
    doc = PDF(filename,
              detect_rotation=True,
              pdf_text_extraction=True)

    extracted_tables = doc.extract_tables(ocr=easyocr,
                                          implicit_rows=False,
                                          borderless_tables=False,
                                          min_confidence=50)

    return extracted_tables

Для простоты будем работать только со следующим списком предметов: экономика, английский язык, математика, биология, информатика и ИКТ. Следующий код менять не нужно.

In [ ]:
target_subjects = ['Экономика', 'Английский язык', 'Математика', 'Биология', 'Информатика и ИКТ']  # рассмотрим только эти предметы

all_dfs = []
response = requests.get('http://rcro.tomsk.ru/munitsipal-ny-j-e-tap-vsosh-2022-23-arhiv/')  # результаты муниципального этапа за 2022-2023 год
main_table = BeautifulSoup(response.text).find('table').findAll('table')[2]  # основная таблица (см. сайт)

for row in main_table.findAll('tr')[1:]:  # первая строка - шапка таблицы
    cells = row.findAll('td')

    subject = cells[0].text.strip()  # название предмета из таблицы
    if subject not in target_subjects:
        continue

    a_hrefs = cells[-1].findAll('a')
    for a_href in a_hrefs:  # ссылки на файлы
        grade = re.search(r'\d+', a_href.text).group(0)
        href = a_href.get('href')

        ext = get_file_extension(href)
        filename = f"{subject}_{grade}.{ext}"
        download_file(href, filename)  # скачивание файла

        extracted_df = extract_data(subject, grade, filename, ext, verbose=True)  # достать данные из файла
        all_dfs.append(extracted_df)

overall_df = pd.concat(all_dfs, ignore_index=True) # фикс для NaN-ов в случае баллов
# overall_df['баллы'] = overall_df['баллы'].str.replace(',', '.').astype(float)  # конвертация баллов в числовой формат

File downloaded successfully to 'Экономика_9.xlsx'



,предмет,класс,фио,баллы
0,Экономика,9,Зюбин Владислав,84
1,Экономика,9,Кириллов Иван,74
2,Экономика,9,Телицин Артемий,74
3,Экономика,9,Шарабарина Елизавета,72
4,Экономика,9,Кочетов Дмитрий,67
...,...,...,...,...
78,Экономика,9,Мациевский Ярослав,5
79,Экономика,9,Белоусова Софья,0
80,Экономика,9,Бокша Полина,0
81,Экономика,9,Авдеенко Артем,0



File downloaded successfully to 'Экономика_10.pdf'



,предмет,класс,фио,баллы
1,Экономика,10,Соколов Владислав Андреевич,71
2,Экономика,10,Танасейчук Тимофей Владимирович,67
3,Экономика,10,Шашко Максим Дмитриевич,64
4,Экономика,10,Аникин Лев Дмитриевич,63
5,Экономика,10,Сухоруков Николай Максимович,61
...,...,...,...,...
80,Экономика,10,Татаркова Юлия Николаевна,4
81,Экономика,10,Вершинин Антон Сергеевич,4
82,Экономика,10,Плотникова Алена Дмитриевна,4
83,Экономика,10,Алейникова Диана Александровна,2



File downloaded successfully to 'Экономика_11.pdf'



...

,предмет,класс,фио,баллы
1,Информатика и ИКТ,9,Лисок Артем Александрович,381
2,Информатика и ИКТ,9,Лисок Никита Александрович,329
3,Информатика и ИКТ,9,Александров Илья Александрович,303
4,Информатика и ИКТ,9,Ивасенко Илья Данилович,255
5,Информатика и ИКТ,9,Полубоярцев Данил Дмитриевич,252
6,Информатика и ИКТ,9,Куликов Ярослав Глебович,228
7,Информатика и ИКТ,9,Гахов Артем Борисович,204
8,Информатика и ИКТ,9,Локотецкий Александр Анатольевич,200
9,Информатика и ИКТ,9,Чуйко Алексей Игоревич,195
10,Информатика и ИКТ,9,Ефимов Артем Игоревич,161



File downloaded successfully to 'Информатика и ИКТ_10.pdf'



,предмет,класс,фио,баллы
1,Информатика и ИКТ,10,Осипов Вячеслав Витальевич,388
2,Информатика и ИКТ,10,Атимасов Никита Игоревич,286
3,Информатика и ИКТ,10,Ильин Василий Игоревич,274
4,Информатика и ИКТ,10,Будаев Никита Олегович,255
5,Информатика и ИКТ,10,Соколов Владислав Андреевич,250
6,Информатика и ИКТ,10,Борисевич Иван Андреевич,235
7,Информатика и ИКТ,10,Михайлов Павел Алексеевич,233
8,Информатика и ИКТ,10,Жуков Сергей Владимирович,233
9,Информатика и ИКТ,10,Трофимов Юрий Алексеевич,233
10,Информатика и ИКТ,10,Малахов Кирилл Антонович,233



File downloaded successfully to 'Информатика и ИКТ_11.pdf'



,предмет,класс,фио,баллы
1,Информатика и ИКТ,11,Агафонов Артём Александрович,500
2,Информатика и ИКТ,11,Зюбин Константин Сергеевич,500
3,Информатика и ИКТ,11,Зиновьев Алексей Игоревич,461
4,Информатика и ИКТ,11,Разинков Иван Константинович,429
5,Информатика и ИКТ,11,Фокин Степан Дмитриевич,429
6,Информатика и ИКТ,11,Литвинов Никита Владимирович,357
7,Информатика и ИКТ,11,Русанова Маргарита Игоревна,337
8,Информатика и ИКТ,11,Богданова Олеся Александровна,320
9,Информатика и ИКТ,11,Горбатенко Александр Александрович,318
10,Информатика и ИКТ,11,Малютин Артём Антонович,318


Должно получиться что-то похожее:

In [ ]:
overall_df['баллы'] = overall_df['баллы'].fillna(0)
overall_df['баллы'] = overall_df['баллы'].apply(lambda val: float(str(val).replace(',', '.')))
overall_df

,предмет,класс,фио,баллы
0,Экономика,9,Зюбин Владислав,84.0
1,Экономика,9,Кириллов Иван,74.0
2,Экономика,9,Телицин Артемий,74.0
3,Экономика,9,Шарабарина Елизавета,72.0
4,Экономика,9,Кочетов Дмитрий,67.0
...,...,...,...,...
2051,Информатика и ИКТ,11,Писаров Александр Михайлович,81.0
2052,Информатика и ИКТ,11,Фрейдман Павел Александрович,80.0
2053,Информатика и ИКТ,11,Николаева Анастасия Александровна,70.0
2054,Информатика и ИКТ,11,Матвеева Юлия Алексеевна,9.0


Постройте сводную таблицу - по каждому предмету для каждого класса посчитайте минимальный балл, средний балл, медианный балл, а также максимальный балл.

Посмотрите на результаты по информатике. О чём могут свидетельствовать подобные цифры?

In [ ]:
# PUT YOUR CODE HERE

new = overall_df.groupby(['предмет', 'класс'])
new.agg({'баллы': ['min', 'mean', 'median', 'max']})

баллы                          
                          min        mean median    max
предмет           класс                                
Английский язык   10      0.0   24.628205   24.0   54.0
                  11      0.0   26.286111   28.0   46.0
                  9       1.0   21.319643   23.0   45.0
Биология          10      0.0   31.925000   31.0   66.0
                  11      0.0   37.262230   36.2   71.0
                  9       7.0   24.850227   25.0   55.0
Информатика и ИКТ 10      0.0  139.406780  123.0  388.0
                  11      0.0  202.830189  168.0  500.0
                  9       0.0  126.861111  110.5  381.0
Математика        10      0.0    4.201754    1.0   26.0
                  11      0.0    6.160194    7.0   28.0
                  8       0.0    2.903670    2.0   28.0
                  9       0.0    6.703030    7.0   35.0
Экономика         10      1.0   26.863095   23.0   71.0
                  11      4.0   32.058586   32.0   69.0
                  9       0.0   28.518072   23.0   84.0

Такие результаты по информатике могут свидетельствовать о том, что большинство ребят не подготовленны к заданиям олимпиады такого уровня, особенно в 9 и 10 классах. В 11 классе люди начинают готовиться к ЕГЭ и набирают какую-то базу знаний, которая улучшает результат, но все равно является недостаточной для успешной сдачи олимпиады. Также благодаря имеющимся данным можно сказать, что в школе не дают достаточное количество материалов для успешной сдачи олимпиады, а скорее всего ученики изучают базовые конструкции для решения типовых заданий без возможности подумать самим.
